In [ ]:
import os
import os
from huggingface_hub import login

# Load from environment variable (secure method)
hf_token = os.getenv("HUGGINGFACE_API_ACCESS")
login(hf_token)

In [ ]:
pip install -U langchain-huggingface

In [ ]:
!pip install -q \
    langchain \
    langchain-community \
    langchain-core \
    langchain-text-splitters \
    langchainhub \
    chromadb \
    huggingface_hub \
    sentence-transformers \
    youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# step2 document Ingestion
# video_id = "cTP58McPp9w"
# try:
#      # If you don’t care which language, this returns the “best” one
#     transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])
#     transcript = " ".join(chunk["text"] for chunk in transcript_list)
#     print("Transcript Loaded ✅")
#     documents = [Document(page_content=transcript, metadata={"source": f"https://youtube.com/watch?v={video_id}"})]
# except TranscriptsDisabled:
#     print("No captions available for this video.")
#     documents = []

# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# chunks = splitter.split_documents(documents)

# step2 document Ingestion
video_id = "kvRjNm4rVBE"
try:
     # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Check if transcript_list is empty
    if not transcript_list:
        print(f"No English captions available for video ID: {video_id}.")
        documents = []
    else:
        transcript = " ".join(chunk["text"] for chunk in transcript_list)
        print(transcript)
        print("Transcript Loaded ✅")
        documents = [Document(page_content=transcript, metadata={"source": f"https://youtube.com/watch?v={video_id}"})]

except TranscriptsDisabled:
    print("No captions available for this video.")
    documents = []
except Exception as e:
    # Catch any other potential errors during transcript retrieval
    print(f"An error occurred while fetching the transcript: {e}")
    documents = []


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

hey everyone welcome back to the channel i hope you guys are doing extremely well so today we will be learning about multiple recursion calls till now you have seen recursion where we had only single uh recursion calls basically we were learning something as there was a function and inside this function only a single function was called again so over here we will be learning something of this pattern where probably we will have a function and there will be multiple function calls yes this is two times so the function is called twice or thrice or four times so you'll be learning multiple recursion calls now generically in all the problems that we'll be solving there might be two times three times four times and so on so in this video i'll be teaching you how to how to have two multiple recursion calls and going forward you will be seeing problems like end queen and other problems where you'll see multiple not only two n recursion calls being happening inside a function okay so to unders

In [ ]:
# Vector_store
# Step 4: Clear old vectorstore (optional but safe)
# import shutil
# persist_directory = "./chroma_store"
# if os.path.exists(persist_directory):
#     shutil.rmtree(persist_directory)
#     print("Old vectorstore cleared ♻️")

# # Step 5: Create new vectorstore
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# vectorstore = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory=persist_directory)
# vectorstore.persist()

InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
import os
import shutil
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# ✅ Use a guaranteed writable temp path
persist_directory = "/tmp/chroma_store"
shutil.rmtree(persist_directory, ignore_errors=True)
os.makedirs(persist_directory, exist_ok=True)
print(f"✅ Old vectorstore cleared and new folder created at {persist_directory}")

# ✅ Make sure 'chunks' is already defined before running this cell
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    chunks,
    embedding=embedding_model,
    persist_directory=persist_directory
)
vectorstore.persist()
print("✅ New vectorstore created and persisted successfully 🎉")


✅ Old vectorstore cleared and new folder created at /tmp/chroma_store
✅ New vectorstore created and persisted successfully 🎉


In [ ]:
all_ids = vectorstore._collection.get()['ids']
for doc_id in all_ids:
    doc = vectorstore._collection.get(ids=[doc_id])['documents']
    print(f"Doc ID: {doc_id}, Content: {doc}")

Doc ID: 78c5f8f4-4b8b-45d1-8b93-ef792f42c52a, Content: ["hey everyone welcome back to the channel i hope you guys are doing extremely well so today we will be learning about multiple recursion calls till now you have seen recursion where we had only single uh recursion calls basically we were learning something as there was a function and inside this function only a single function was called again so over here we will be learning something of this pattern where probably we will have a function and there will be multiple function calls yes this is two times so the function is called twice or thrice or four times so you'll be learning multiple recursion calls now generically in all the problems that we'll be solving there might be two times three times four times and so on so in this video i'll be teaching you how to how to have two multiple recursion calls and going forward you will be seeing problems like end queen and other problems where you'll see multiple not only two n recursion 

In [ ]:
# Step 4: Retriever
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 10})

In [ ]:
question = "What is the meaning of life?"
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(metadata={'source': 'https://youtube.com/watch?v=kvRjNm4rVBE'}, page_content="you are returning last plus second last so remember this recursion call is being done so our stack space will wait over here like we will be waiting at this line we will not be going beneath this line we will be waiting the stack space will keep this function saying that okay i'm gonna wait till this doesn't gives me this answer perfect now f of three will be like okay this is not executed f of two will be executed so again f of two will be executed and it will be like let's call f of 2 so the if line is not executed last will be f of what will be last f 1 will be second last f 0 correct so and then they can return last plus second last so if i'm calling last f of one will be called so whenever f of one is called if n lesser than equal to one gets executed you need to understand if n lesser than equal to one gets executed and it returns the value n so this basically gives you a value one because tha

In [ ]:
#step 4 Augmentation
llm =  HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task = "text-generation",
    temperature = 0.5
    )

In [ ]:
prompt = PromptTemplate(
    template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context below.
If the context is insufficient, say "I don't know".

Transcript:
{context}

Question: {question}
Answer:""",
    input_variables=["context", "question"]
)

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parser = StrOutputParser()
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('is this video discused about recursion tree')

' Yes, the video discusses recursion trees and how they help in understanding multiple recursion calls. The video explains how the time complexity of multiple recursion calls is near about 2 to the power n. The video also demonstrates how to draw recursion trees for multiple recursion calls using the example of the Fibonacci sequence.'